In [1]:
from arquivos import central
import matplotlib.pyplot as plt
from numba import njit
import numpy as np
import pandas as pd

In [2]:
robot = central.Central()

In [3]:
robot.get_normal_data_csv('C:\\Users\\johnk\\Google Drive\\My_Project\\Dados\\Data_D1_3k_icmarkets.csv', drop=True, drop_list=['Unnamed: 0'])

Dados Carregados com Sucesso.


In [4]:
%%time
robot.pct_data()
robot.balance_signal4()
df = robot.back_tpsl(multiply_tp=3, multiply_sl=1,analyse=False, multi_test=False, plot=False)

Wall time: 6.77 s


In [47]:
df

,Result,Short_Trades,Long_Trades,EURCHF,EURGBP,EURJPY,EURNZD,EURUSD,EURAUD,EURCAD,...,AUDJPY,CADJPY,CHFJPY,NZDJPY,AUDCHF,CADCHF,NZDCHF,AUDNZD,NZDCAD,AUDCAD
0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
1,1011.675842,994.489685,1011.675842,1011.675842,993.676758,991.193237,992.093689,994.489685,1025.773682,992.883667,...,999.928528,999.928406,999.929138,999.927124,1034.035522,988.345459,977.923706,996.683960,988.020447,994.050659
2,1006.165527,988.547058,1007.860962,1023.062866,987.365967,1017.984558,1015.990356,988.988953,1017.038757,1014.342712,...,999.856934,999.863647,999.858215,999.865845,1065.501587,976.649597,955.280762,993.345764,975.358704,987.966797
3,1000.222900,986.188904,1004.815063,1034.442749,981.034119,1008.974121,1007.914978,1005.486816,1042.311646,1007.060730,...,999.791809,999.791138,999.787292,999.793091,1097.712036,1011.571472,1022.194885,990.081238,1013.835327,1006.481995
4,996.408020,978.393921,1038.850586,1046.346313,999.996338,997.993103,999.882263,999.854065,1033.627563,1029.100708,...,999.719727,999.718811,999.720093,999.720520,1087.075684,1000.203918,1001.048828,985.533386,997.651489,997.816650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10879,14830.786133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10880,14845.684570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10881,14841.569336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10882,14841.499023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
365 / 2500

0.146

In [15]:
250 / 2500

0.1

In [20]:
c_inicial = 1000
c_final = 12395
c = c_final / c_inicial

In [21]:
c

12.395

In [26]:
(c-1) ** 0.10 -1 

0.2754734796588414

In [5]:
df.mean(axis=1)

0     0.145467
1     0.791522
2     0.780550
3     0.951304
4     1.039089
5     1.274672
6     1.033771
7     1.023664
8     1.015980
9     1.005918
10    1.052712
11    1.120028
12    1.185617
13    1.166923
14    1.321257
15    1.219015
16    1.321329
17    1.262347
18    1.278418
19    1.296211
20    1.193387
21    1.271187
22    1.287158
23    1.217902
24    1.230640
25    1.172203
26    1.199870
27    1.249921
28    1.177329
dtype: float64

In [11]:
df.mean(axis=1)

0     0.061214
1     0.733079
2     0.722818
3     0.862569
4     0.978071
5     1.186888
6     0.983440
7     0.972688
8     0.963153
9     0.950559
10    0.981249
11    1.042553
12    1.086036
13    1.066064
14    1.186229
15    1.103396
16    1.194881
17    1.147490
18    1.152229
19    1.170076
20    1.091553
21    1.148201
22    1.160103
23    1.112065
24    1.127709
25    1.082955
26    1.107120
27    1.146534
28    1.090236
dtype: float64

In [5]:
robot.get_normal_data().iloc[robot.index_list_wfa[0][0]]

,EURCHF_Open,EURCHF_Close,EURCHF_High,EURCHF_Low,EURGBP_Open,EURGBP_Close,EURGBP_High,EURGBP_Low,EURJPY_Open,EURJPY_Close,...,CADCHF_sell,CADCHF_buy,NZDCHF_sell,NZDCHF_buy,AUDNZD_sell,AUDNZD_buy,NZDCAD_sell,NZDCAD_buy,AUDCAD_sell,AUDCAD_buy
0,1.14620,1.14167,1.15173,1.13983,0.88087,0.87562,0.88222,0.87368,111.043,111.360,...,False,False,False,False,False,False,False,False,False,False
450,1.22744,1.23321,1.23444,1.22558,0.85351,0.85682,0.85860,0.85257,124.360,127.267,...,True,False,True,False,True,False,False,True,False,True


In [32]:
len(robot.get_normal_walk_forward())

27

In [43]:
len(robot.get_normal_walk_forward()[2][0][0])

450

In [31]:
robot.index_list_wfa[0]

[[0, 450], [450, 600]]

In [11]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.131585,0.471500,0.155050,0.098004,0.557634,-0.582576,0.193060,-1.656168,-0.022313,-0.040405,-0.163772,0.133289,-0.113432
1,-0.106811,0.601169,0.307267,0.487373,0.304664,-0.016300,0.194920,-1.070203,0.492979,0.373153,0.489928,0.180637,0.158533
2,0.009260,0.681710,0.334878,0.259896,0.405998,0.066248,0.195401,-0.848969,0.391458,0.134335,0.443975,0.219990,0.206248
3,-0.013904,0.702777,0.169681,0.470282,0.494422,0.059418,0.424974,-0.821523,0.257382,0.157430,0.270617,0.511288,0.173364
4,0.144640,0.671634,0.128711,0.337433,0.395393,0.248699,0.457221,-0.383082,0.231023,0.130819,0.690158,0.437285,0.204244
5,0.127902,0.774042,0.202346,0.306388,0.563115,0.245470,0.260696,-0.110946,0.221537,0.106756,1.019085,0.487118,0.280851
6,0.124698,0.708443,0.216848,0.311278,0.335127,0.530941,0.268297,0.057341,0.146549,0.020028,0.473001,0.397635,0.257662
7,0.138075,0.660596,0.212733,0.344042,0.449169,0.453946,0.284367,-0.092375,0.160595,-0.005262,0.416636,0.321860,0.417052
8,0.108219,0.642009,0.189824,0.311997,0.431596,0.413101,0.434838,-0.169500,0.136617,0.104740,0.261359,0.389397,0.437935
9,0.063690,0.621124,0.335946,0.343131,0.451383,0.412602,0.407901,-0.135657,0.134488,0.105528,0.029521,0.406154,0.441130
